<a href="https://colab.research.google.com/github/hadwin-357/ProteinMPNN_breakdown/blob/main/testing_model_utils_2_loss_nll.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# testing_model_util_functions
# loss_null
'''
inputs-  S ground truth, log_probs: calculated logprob, mask: position mask(if masked 1, else 0))


Function explain:
compare to loss_smooth(used in training), this functioni is used in final step to calculate loss (so regularization is not needed)
use NLLLoss
reshape log_probs and S into 2D to fit NLLLoss inputs requirement to calculate loss
S_argmaxed: output predicted logits
true_false: 1 or true, 0.0 for false
calculate average loss for masked position
caclaculate average loss
'''

'\ninputs-  S ground truth, log_probs: calculated logprob, mask: position mask(if masked 1, else 0))\n\n\nFunction explain:\ncompare to loss_smooth(used in training), this functioni is used in final step to calculate loss (so regularization is not needed)\nuse NLLLoss \nreshape log_probs and S into 2D to fit NLLLoss inputs requirement to calculate loss\nS_argmaxed: output predicted logits\ntrue_false: 1 or true, 0.0 for false\ncalculate average loss for masked position\ncaclaculate average loss\n'

In [2]:
import torch

In [3]:
def loss_nll(S, log_probs, mask):
    """ Negative log probabilities """
    criterion = torch.nn.NLLLoss(reduction='none')
    loss = criterion(
        log_probs.contiguous().view(-1, log_probs.size(-1)), S.contiguous().view(-1)
    ).view(S.size())
    S_argmaxed = torch.argmax(log_probs,-1) #[B, L]
    true_false = (S == S_argmaxed).float()
    loss_av = torch.sum(loss * mask) / torch.sum(mask)
    return loss, loss_av, true_false

In [4]:
# break down
# S is the ground truth


# Example ground truth labels
# Suppose you have a batch size of 4 and sequence length of 10
batch_size = 4
sequence_length = 10
num_classes = 21  # Assuming 21 classes

# Create random ground truth labels
S = torch.randint(0, num_classes, (batch_size, sequence_length))

# Verify the shape and data type of S
print("Shape of S:", S.shape)
print("Data type of S:", S.dtype)

Shape of S: torch.Size([4, 10])
Data type of S: torch.int64


In [5]:
# Create random logits (predicted probabilities)
log_probs = torch.randn(batch_size, sequence_length, num_classes)

# Verify the shape and data type of logits
print("Shape of log_probs:", log_probs.shape)
print("Data type of log_probs:", log_probs.dtype)

Shape of log_probs: torch.Size([4, 10, 21])
Data type of log_probs: torch.float32


In [6]:
log_probs.contiguous().view(-1, log_probs.size(-1)).shape

torch.Size([40, 21])

In [7]:
S.contiguous().view(-1).shape

torch.Size([40])

In [8]:
criterion = torch.nn.NLLLoss(reduction='none')
loss = criterion(
    log_probs.contiguous().view(-1, log_probs.size(-1)), S.contiguous().view(-1)
).view(S.size())

In [9]:
loss.shape

torch.Size([4, 10])

In [10]:
loss[0,:]  # loss of each residues in batch 0

tensor([ 0.1130, -0.4759, -1.5217,  0.3597,  0.0932,  0.2528,  1.6670,  1.2122,
        -0.2960,  1.3142])

In [12]:
S_argmaxed = torch.argmax(log_probs,-1) #[B, L]
S_argmaxed.shape

torch.Size([4, 10])

In [13]:
S_argmaxed[0,:] # [predicted logits/residue class

tensor([18,  2, 20, 17,  0, 15, 12, 17,  4,  0])

In [14]:
true_false = (S == S_argmaxed).float()
true_false[0,:]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])